# Objet : Test opendata des données de qualité de l'air

## Objectif

- valider sur des cas réels l'outil de traitement des "listes indexées"
- identifier les apports que pourraient avoir ce type d'outil

## Résultats
- interêt d'utiliser les outils Pandas
- à compléter

## Usages possibles 
- à compléter

## Autres points
- chargement sur MongoDB à tester
- Outil de requète à tester

données utilisées : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2022/

------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [1]:
from pprint import pprint
from collections import Counter
from time import time
import csv
from util import util
from observation import Ilist, Iindex
from copy import copy
import pandas as pd

chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/air/data_lcsqa/'

In [2]:

data = []
t0 = time()
nb_fichiers = 1
annee = 2022
mois = 1
jour = 1
for i in range(nb_fichiers):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour+i, '02d') +'.csv'
    data.append(pd.read_csv(file, sep=';'))
data2 = pd.concat(data, ignore_index=True, join='inner')
data2[['Date de début','Date de fin']] = data2[['Date de début','Date de fin']].astype('datetime64')
data2 = data2.astype('category')

data2.pop('valeur brute')
print('data2 : \n', len(data2), '\n', list(data2), '\n', time()-t0)


data2 : 
 49392 
 ['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', 'Réglementaire', "type d'évaluation", 'procédure de mesure', 'type de valeur', 'valeur', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité'] 
 11.279568433761597


----
## initialisation de l'objet Ilist
- l'initialisation pourrait être automatisée à partir du fichier csv
- identification de 64 775 valeurs différentes sur un total de 11 163 x 49 valeurs ("taux d'unicité" de 12%)
- la taille minimale serait de 1,4 Mo (données csv "quotées") pour un maximum de 9,6 Mo (données csv "quotées")

In [3]:
listidx = data2
listidx.__class__.__name__ == 'DataFrame'
for name, idx in listidx.astype('category').items():
    lis = list(idx.cat.categories)
    print(len(lis))
    

24
24
18
70
70
532
532
5
9
3
26
1
4
58
1
1956
3
0
0
0
3
2


In [4]:
t0=time()
idxs2 = Ilist.obj(data2)
#idxs2.lindex[0].set_codec(util.castobj(idxs2.lindex[0].codec, 'DatationValue'))
#idxs2.lindex[1].set_codec(util.castobj(idxs2.lindex[1].codec, 'DatationValue'))
print('idxs (len, lenlidx, sumcodec) : ', len(idxs2), len(idxs2.idxlen), sum(idxs2.idxlen), time()-t0)
#idxs2.setvar('valeur')
idxs3 = Ilist(idxs2)
print(idxs3 == idxs2)

idxs (len, lenlidx, sumcodec) :  49392 22 3344 3.7323484420776367
True


In [ ]:
from observation import DatationValue
print(idxs3.lindex[0].codec[0])
print(type(idxs3.lindex[0].codec[0]))
print(DatationValue(idxs3.lindex[0].codec[0]))


In [ ]:
t0=time()
print(idxs2.category, '\n')
print(idxs2.tree(), '\n')
print(idxs2.tree(mode='distance'))
print(time()-t0)

## formats de base

In [5]:
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='full')
fullsize = len(js)
print('fullsize', len(js), time()-t0)

fullsize 14988223 3.054269790649414


In [6]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 49392 12.60580039024353
controle égalité : True 0.3321690559387207


In [7]:
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='nokeys')
minsize = len(js)
print('minsize', len(js), time()-t0)

t0=time()
js = idxs2.to_obj(encoded=True, encode_format='cbor', modecodec='nokeys')
print('mincborsize', len(js), time()-t0)


minsize 34498 0.012923717498779297
mincborsize 33259 0.016391277313232422


----
## format default
- 

In [8]:
champ = idxs2.nindex
'''champ('valeur').tostdcodec(inplace=True)
idxs2.setcanonorder()
pprint(idxs2.category)'''
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='default')
defaultsize = len(js)
print('defaultsize : ', len(js), time()-t0, '\n')
print('indicator default : ', idxs2.indicator(fullsize, defaultsize), '\n')

t0=time()
pprint(champ('code site').couplinginfos(champ('Date de début')))
print('\n', idxs2.tree(mode='diff'))
print('\nanalyse : ', time()-t0)

defaultsize :  3832752 0.12758207321166992 

indicator default :  {'total values': 1136016, 'mean size': 13.194, 'unique values': 3366, 'mean coding size': 3.345, 'unicity level': 0.003, 'optimize level': 0.256, 'object lightness': 0.254, 'maxgain': 0.997, 'gain': 0.744} 

{'diff': 508,
 'dist': 12768,
 'distance': 12744,
 'distmax': 12768,
 'distmin': 532,
 'distrate': 1.0,
 'disttomax': 0,
 'disttomin': 12236,
 'rate': 1.0,
 'typecoupl': 'crossed'}

 -1: root-diff (49392)
   11: Réglementaire (0.00e+00 - 1)
   14: type de valeur (0.00e+00 - 1)
   15: valeur (1.00e+00 - 1956)
      0 : Date de début (2.67e-01 - 24)
         1 : Date de fin (0.00e+00 - 24)
      5 : code site (3.45e-02 - 532)
         2 : Organisme (0.00e+00 - 18)
         3 : code zas (0.00e+00 - 70)
            4 : Zas (0.00e+00 - 70)
         6 : nom site (7.08e-06 - 532)
         7 : type d implantation (0.00e+00 - 5)
         9 : type d influence (1.32e-02 - 3)
         10: discriminant (3.87e-02 - 26)
         12

In [9]:
idxs4 = copy(idxs2)
champ = idxs4.nindex

In [10]:
print(idxs2.lname)

['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', 'Réglementaire', "type d'évaluation", 'procédure de mesure', 'type de valeur', 'valeur', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité']


In [11]:
print(idxs2.analysis.getmatrix(['Polluant', 'unité de mesure']))
notcoupl = champ('Polluant').coupling(champ('unité de mesure'), derived=True)
print('nombre de non couplés : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = [(champ('Polluant')[i], champ('unité de mesure')[i]) for i in notcoupl[:2000]]
print('liste')
pprint(set(liste), width=120)

{'dist': 16, 'distrate': 0.3888888888888889, 'disttomin': 7, 'disttomax': 11, 'distmin': 9, 'distmax': 27, 'diff': 6, 'distance': 13, 'rate': 0.5416666666666666, 'typecoupl': 'link'}
nombre de non couplés :  48840

liste des premières incohérences : 
liste
{('O3', 'µg-m3'), ('O3', 'µg/m3')}


In [12]:
print(idxs2.analysis.getmatrix(['code site', 'nom site']))
notcoupl = champ('code site').coupling(champ('nom site'), derived=False)
print('nombre de non couplés : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = []
for i in range(len(notcoupl)): liste.append((champ('code site')[notcoupl[i]], champ('nom site')[notcoupl[i]]))
pprint(set(liste), width=120)

{'dist': 534, 'distrate': 7.079846508927687e-06, 'disttomin': 2, 'disttomax': 282490, 'distmin': 532, 'distmax': 283024, 'diff': 0, 'distance': 2, 'rate': 7.079846508927687e-06, 'typecoupl': 'link'}
nombre de non couplés :  168

liste des premières incohérences : 
{('FR19053', 'QUIMPER ZOLA'), ('FR19053', 'Quimper Zola'), ('FR19007', 'HALLES'), ('FR19007', 'Rennes Les Halles')}


In [13]:
print(idxs2.analysis.getmatrix(['code site', 'nom site']))
champ = idxs2.nindex 
notcoupl = champ('nom site').coupling(champ('code site'), derived=False)
print('nombre de non couplés : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = []
for i in range(len(notcoupl)): liste.append((champ('code site')[notcoupl[i]], champ('nom site')[notcoupl[i]]))
pprint(set(liste), width=120)

{'dist': 534, 'distrate': 7.079846508927687e-06, 'disttomin': 2, 'disttomax': 282490, 'distmin': 532, 'distmax': 283024, 'diff': 0, 'distance': 2, 'rate': 7.079846508927687e-06, 'typecoupl': 'link'}
nombre de non couplés :  384

liste des premières incohérences : 
{('FR33101', 'PASTEUR'), ('FR23078', 'SAINT EXUPERY'), ('FR20048', 'SAINT EXUPERY'), ('FR23004', 'PASTEUR')}


In [14]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 49392 3.7000155448913574
True
controle égalité : True 0.330366849899292


----
## Format optimisé
- 

In [15]:
idxs4.reindex()
idxs4.coupling(param='distance', level=500)
print(idxs4.tree())
t0=time()
js = idxs4.to_obj(modecodec='optimize', encoded=True)
optimizesize = len(js)
print('optimizesize : ', optimizesize, time()-t0, '\n')
print('indicator optimize : ', idxs2.indicator(fullsize, optimizesize), '\n')

t0=time()
js = idxs4.to_obj(encoded=True, modecodec='optimize', encode_format='cbor')
cborsize = len(js)
print('cborsize : ', cborsize, time()-t0, '\n')
print('indicator cbor : ', idxs2.indicator(fullsize, cborsize))

-1: root-derived (49392)
   0 : Date de début (24)
      1 : Date de fin (24)
   5 : code site (2616)
      3 : code zas (2143)
         2 : Organisme (18)
         4 : Zas (70)
         13: procédure de mesure (1299)
            8 : Polluant (310)
               7 : type d implantation (101)
                  12: type d évaluation (49)
                     16: unité de mesure (21)
                        9 : type d influence (9)
                           20: code qualité (3)
                              21: validité (2)
            10: discriminant (26)
      6 : nom site (2616)
   11: Réglementaire (1)
   14: type de valeur (1)
   15: valeur (1956)
   17: taux de saisie (1)
   18: couverture temporelle (1)
   19: couverture de données (1)
optimizesize :  933230 0.35938000679016113 

indicator optimize :  {'total values': 1136016, 'mean size': 13.194, 'unique values': 3370, 'mean coding size': 0.785, 'unicity level': 0.003, 'optimize level': 0.062, 'object lightness': 0.059, 'maxgai

In [16]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 49392 3.2341301441192627
True
controle égalité : True 0.40999436378479004


----
## Format BD
- 

In [17]:
t0=time()
js = idxs2.to_obj(modecodec='dict', encoded=True)
dictsize = len(js)
print('dictsize : ', dictsize, time()-t0, '\n')
print('indicator dict : ', idxs2.indicator(fullsize, dictsize), '\n')

dictsize :  7472116 4.18480658531189 

indicator dict :  {'total values': 1136016, 'mean size': 13.194, 'unique values': 3370, 'mean coding size': 6.558, 'unicity level': 0.003, 'optimize level': 0.499, 'object lightness': 0.497, 'maxgain': 0.997, 'gain': 0.501} 



In [18]:
t0=time()
new = Ilist.from_obj(js)
print('new', len(new), time()-t0)
print(idxs3 == idxs2)
t0=time()
verif = new == idxs2
print('controle égalité :', verif, time()-t0)

new 49392 3.8931832313537598
True
controle égalité : True 0.34420299530029297


In [19]:
idxs=Ilist.ext(idxs2[0:5], idxs2.lname)
print(idxs.to_obj())

[['Date de début', ['2022-01-01T02:00:00+00:00', '2022-01-01T00:00:00+00:00', '2022-01-01T03:00:00+00:00', '2021-12-31T23:00:00+00:00', '2022-01-01T01:00:00+00:00'], [3, 1, 4, 0, 2]], ['Date de fin', ['2022-01-01T03:00:00+00:00', '2022-01-01T01:00:00+00:00', '2022-01-01T04:00:00+00:00', '2022-01-01T00:00:00+00:00', '2022-01-01T02:00:00+00:00'], 0], ['Organisme', ['ATMO GRAND EST']], ['code zas', ['FR44ZAG02']], ['Zas', ['ZAG METZ']], ['code site', ['FR01011']], ['nom site', ['Metz-Centre']], ["type d'implantation", ['Urbaine']], ['Polluant', ['NO']], ["type d'influence", ['Fond']], ['discriminant', ['A']], ['Réglementaire', ['Oui']], ["type d'évaluation", ['mesures fixes']], ['procédure de mesure', ['Auto NO Conf meth CHIMILU']], ['type de valeur', ['moyenne horaire validée']], ['valeur', [1.7, 1.4, 1.2, 1.5, 1.6], 0], ['unité de mesure', ['µg-m3']], ['taux de saisie', [-1]], ['couverture temporelle', [-1]], ['couverture de données', [-1]], ['code qualité', ['A']], ['validité', [1]]]


synthèse:

1  fichier  : full  14.4, def  3.8, opt 0.9 cbor 0.3 dic 7.5    500     
3  fichiers : full  43.1, def 11.5, opt 2.5 cbor 1.3 dic 23.7   500    
5  fichiers : full  71.9, def 19.3, opt 4.1 cbor 2.1 dic 41.1   500    
10 fichiers : full 143.7, def 39.0, opt 8.2 cbor 4.0 dic 84.5   500         493 225 lignes

In [20]:
from test_mongo import clientMongo
from observation import Observation, ESSearch

collec = clientMongo()['air_quality']['lcsqa']

# envoi de l'objet 'idxs' (l'objet Observation ajoute des méta-donnnées à l'objet Ilist) dans une base mongoDB
json_lcsqa = Observation(idxs, name='01/01/2022').json(modecodec='dict')
print(collec.insert_one(json_lcsqa).inserted_id)

'''# définition du critère de recheche de l'objet 'tarif'
critere_tarif = {'path':'name', 'operand': 'example_tarif', 'comparator': '=='}

# vérification que l'objet récupéré (tarif_complet remis au format Ilist) est bien identique à celui envoyé
tarif_complet = ESSearch(critere_tarif, collec).execute(single=True)
print(Ilist(tarif_complet) == tarif)

#ajout d'un critère supplémentaire pour ne cherche que la partie de 'tarif' qui concerne les aliments de type 'legume'
tarif_legume = ESSearch([['aliment', 'legume'], critere_tarif], collec).execute(single=True)
print(Ilist(tarif_legume))'''

63b763bf29a458ea8438f8ee


"# définition du critère de recheche de l'objet 'tarif'\ncritere_tarif = {'path':'name', 'operand': 'example_tarif', 'comparator': '=='}\n\n# vérification que l'objet récupéré (tarif_complet remis au format Ilist) est bien identique à celui envoyé\ntarif_complet = ESSearch(critere_tarif, collec).execute(single=True)\nprint(Ilist(tarif_complet) == tarif)\n\n#ajout d'un critère supplémentaire pour ne cherche que la partie de 'tarif' qui concerne les aliments de type 'legume'\ntarif_legume = ESSearch([['aliment', 'legume'], critere_tarif], collec).execute(single=True)\nprint(Ilist(tarif_legume))"